In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
# from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

c:\Users\chris\.conda\envs\rag2\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [3]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and informative. She's always willing to help students outside of class."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant instructor! Prof. Chen makes complex algorithms easy to understand. His projects are challenging but rewarding.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Dr. Thompson knows her subject well, but her lectures can be a bit dry. More interactive sessions would be helpful.'},
 {'professor': 'Prof. David Martinez',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Martinez brings history to life with his storytelling. His exams are tough but fair.'},
 {'professor': 'Dr. Rachel Kim',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. Kim is an exceptional teacher. Her lab sessions are well-organized and she explains concepts cl

In [8]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")
print(torch.__version__)
print(torch.version.cuda)
torch.cuda.empty_cache()

True
NVIDIA GeForce RTX 2080 SUPER
2.4.0
12.4


In [9]:
processed_data = []
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()

for review in data['reviews']:
    embedding = model.encode([review])
    processed_data.append({
        "values": embedding[0],
        "id": review['professor'],
        "metadata": {
            "review": review['review'],
            "subjects": review['subject'],
            "stars": review['stars'],
        }
    })

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
processed_data[1]

{'values': array([-1.039665  ,  0.15772526,  1.5772394 , ..., -0.14516646,
        -0.13970429,  3.0700932 ], dtype=float32),
 'id': 'Prof. Michael Chen',
 'metadata': {'review': 'Brilliant instructor! Prof. Chen makes complex algorithms easy to understand. His projects are challenging but rewarding.',
  'subjects': 'Computer Science',
  'stars': 5}}